In [0]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.1)


In [0]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup 
from gensim.models import Word2Vec

In [0]:
import nltk
nltk.download('punkt')
#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
df = pd.read_csv('/content/data.csv')

In [0]:
df.head

<bound method NDFrame.head of           Make       Model  Year  ... city mpg  Popularity   MSRP
0          BMW  1 Series M  2011  ...       19        3916  46135
1          BMW    1 Series  2011  ...       19        3916  40650
2          BMW    1 Series  2011  ...       20        3916  36350
3          BMW    1 Series  2011  ...       18        3916  29450
4          BMW    1 Series  2011  ...       18        3916  34500
...        ...         ...   ...  ...      ...         ...    ...
11909    Acura         ZDX  2012  ...       16         204  46120
11910    Acura         ZDX  2012  ...       16         204  56670
11911    Acura         ZDX  2012  ...       16         204  50620
11912    Acura         ZDX  2013  ...       16         204  50920
11913  Lincoln      Zephyr  2006  ...       17          61  28995

[11914 rows x 16 columns]>

In [0]:
df['Maker_Model']= df['Make']+" "+df['Model']

In [0]:
df1 = df[['Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size', 'Vehicle Style', 'Maker_Model']]

In [0]:
df1

,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style,Maker_Model
0,premium unleaded (required),MANUAL,rear wheel drive,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,BMW 1 Series M
1,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Convertible,BMW 1 Series
2,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,High-Performance",Compact,Coupe,BMW 1 Series
3,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Coupe,BMW 1 Series
4,premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible,BMW 1 Series
...,...,...,...,...,...,...,...
11909,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,Acura ZDX
11910,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,Acura ZDX
11911,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,Acura ZDX
11912,premium unleaded (recommended),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,Acura ZDX


In [0]:
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)

In [0]:
df2

0        premium unleaded (required),MANUAL,rear wheel ...
1        premium unleaded (required),MANUAL,rear wheel ...
2        premium unleaded (required),MANUAL,rear wheel ...
3        premium unleaded (required),MANUAL,rear wheel ...
4        premium unleaded (required),MANUAL,rear wheel ...
                               ...                        
11909    premium unleaded (required),AUTOMATIC,all whee...
11910    premium unleaded (required),AUTOMATIC,all whee...
11911    premium unleaded (required),AUTOMATIC,all whee...
11912    premium unleaded (recommended),AUTOMATIC,all w...
11913    regular unleaded,AUTOMATIC,front wheel drive,L...
Length: 11914, dtype: object

In [0]:
df2 = pd.DataFrame({'clean': df2})
df2

,clean
0,"premium unleaded (required),MANUAL,rear wheel ..."
1,"premium unleaded (required),MANUAL,rear wheel ..."
2,"premium unleaded (required),MANUAL,rear wheel ..."
3,"premium unleaded (required),MANUAL,rear wheel ..."
4,"premium unleaded (required),MANUAL,rear wheel ..."
...,...
11909,"premium unleaded (required),AUTOMATIC,all whee..."
11910,"premium unleaded (required),AUTOMATIC,all whee..."
11911,"premium unleaded (required),AUTOMATIC,all whee..."
11912,"premium unleaded (recommended),AUTOMATIC,all w..."


In [0]:
sent = [row.split(',') for row in df2['clean']]


In [0]:
sent[0]

['premium unleaded (required)',
 'MANUAL',
 'rear wheel drive',
 'Factory Tuner',
 'Luxury',
 'High-Performance',
 'Compact',
 'Coupe',
 'BMW 1 Series M']

In [0]:
model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [0]:
model.train(sent,total_examples=len(sent),epochs=10)

(298976, 881290)

In [0]:
model.wv.similarity('Toyota ECHO','Toyota ECHO')

1.0

In [0]:
model['Toyota ECHO']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.02605368,  0.09608921,  0.03256813, -0.44634137,  0.14392732,
       -0.03454543,  0.27372572, -0.0010313 ,  0.11087945,  0.08043574,
        0.20806155, -0.19673485, -0.09616522,  0.1313442 , -0.1449942 ,
       -0.04835655, -0.11149673,  0.26558363, -0.27407217,  0.03077132,
       -0.07308191,  0.1344363 ,  0.09768037,  0.08002964,  0.12645838,
       -0.27592814,  0.06502793,  0.16848172, -0.15953445, -0.12898403,
       -0.07367063,  0.12596507,  0.17377824,  0.01179708, -0.22213224,
       -0.05054659,  0.8218265 ,  0.10931011, -0.12076827,  0.09880126,
        0.26956972,  0.07970778, -0.0180229 ,  0.15603133, -0.03801795,
        0.13272755,  0.26956728, -0.08899746,  0.43592757,  0.36966434],
      dtype=float32)

In [0]:
model.wv.similarity('Ford E-Series Van','Mercedes-Benz E-Class')

0.6136378